# Una Introducción a Pandas

Esta libreta es una modificación (sin muchos cambios) de la introducción a Pandas realizada por Julians Evans y que se encuentra dentro de la (documentación oficial de Pandas)[https://github.com/jvns/pandas-cookbook], Unas cosas se redujeron, y otras se dejaron en forma de ejercicios para practicar diferentes cosas.

In [ ]:
# Figuras en linea
import matplotlib
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 5)      # Tamaño de las gráficas

matplotlib.style.use('ggplot')



## 1 Leyendo datos de un archivo csv

Para leer archivos CSV, se utiliza la función `read_csv`. Por default se asume la coma como el indicador de separación. Vamos a ver un conjunto de datos sobre [ciclistas en la ciudad de Montreal](http://donnees.ville.montreal.qc.ca/dataset/velos-comptage). Los cuales se encuentran ya descargados. Esta base de datos describe la cantidad de ciclistas que transitaron por 7 diferentes rutas por día.

In [ ]:
broken_df = pd.read_csv('bikes.csv')

In [ ]:
# Vamos a ver las 3 primeras entradas
broken_df[:3]

Ups! Así, los datos no son útiles. Afortunadamente, `read_csv` viene con un montón de opciones para ajustarse a los diferentes archivos CSV. En particular es necesario:

* cambiar el indicador de columna por  `;`
* Cambiar la codificación por `'latin1'` (por default es `'utf8'`)
* Transformar los datos de la columna `Date` a algun tipo estandar `datetime`  
* Avisar que el estilo de los datos es francés (como en español) esto es día/mes/año
* Hacer que el índice sea la columna `Date`

In [ ]:
fixed_df = pd.read_csv('bikes.csv', sep=';', encoding='utf8', parse_dates=['Date'], dayfirst=True, index_col='Date')
fixed_df[:3]

In [ ]:
fixed_df.describe()

## 2 Seleccionar columnas

In [ ]:
fixed_df['Berri 1']

## 3 Graficar datos

In [ ]:
fixed_df['Berri 1'].plot()

Igualmente se pueden graficar todos los días al mismo tiempo. Para ver mejor habrá que hacer un poco más grande la figura. Si bien es poco lo que se puede sacar de conclusión en este gráfico, se puede observar que el comportamiento de todos los caminos son más o menos parecidos. Igualmente, por las estacones, podríamos asociar de alguna manera el comportaiento de los ciclistas con el clima.

In [ ]:
fixed_df.plot(figsize=(15, 10))

### Ejercicio: Grafica 3 rutas únicamente desde la entrada 130 a la 160 (los días del verano)

In [ ]:
# Escribe aquí tu código

## 4 Extraer una columna y transformar el índice (fecha)

Supongamos ahora que tenemos la siguiente pregunta ¿Que día de la semana la gente circula más por el trayecto *Berri 1* (una ciclovía al lado de una calle principal de Montreal). Bueno, lo que tendríamos que hacer es extraer la columna que nos interesa como un nuevo *DataFrame* y luego agregarle una nueva variable como es el día de la semana.

Primero copiemos físicamente la coluna que nos interesa

In [ ]:
berri_bikes = fixed_df[['Berri 1']].copy()
berri_bikes.head()

Ahora tenemos que malipular las fechas, las cuales se encuentran en el índice del *DataFrame*

In [ ]:
berri_bikes.index

Curioso que lo que se registró solo llego hasta el 5 de noviembre del 2012 ¿Verdad? Ahora podemos usar las facilidades que nos dá Pandas para manipular fechas. Por ejemplo, si queremos el día del mes:

In [ ]:
berri_bikes.index.day

y si queremos el día de la semana:

In [ ]:
berri_bikes.index.weekday

Ahora agregamos una nueva variable a nuestro *DataFrame* de manera bastante intuitiva

In [ ]:
berri_bikes.loc[:, u'Día de la semana'] = berri_bikes.index.weekday
berri_bikes.head()

## 5 Agregación de información

Ahora nos falta agregar la información, y para eso vamos a utilizar el método `.groupby` de la clase *DataFrame*, el cual es muy parecido al que se usa en SQL. Veamos como se utiliza en forma básica, pero revisa la [documentación](http://pandas.pydata.org/pandas-docs/stable/groupby.html), ya que es una herramienta muy poderosa para el análisis y tratamiento de los datos.

In [ ]:
paseos_por_dia = berri_bikes.groupby(u'Día de la semana').aggregate(sum)
paseos_por_dia

¿Días de la semana de 0 a 6? Eso no tiene mucho sentido si queremos seguir usando esta información para análisis posteriores. Es una buena práctica procurar que todas las columnas tengan nombres significativos, y que los valores se codifiquen de una manera que sean claros.

In [ ]:
paseos_por_dia.index = ['Lunes', 'Martes', u'Miércoles', 'Jueves', 'Viernes', u'Sábado', 'Domingo']
paseos_por_dia

¡Mejor! Y mucho mejor si lo visualizamos como una gráfica:

In [ ]:
paseos_por_dia.plot(kind='bar')

Alguien podría argumentar que las cuentas totales no es la mejor manera que representar lo que pasa en cada día, porque la variabilidad podría ser factor importante. Como la observación nos parece válida, hagámos el análisis de otra manera.

In [ ]:
berri_bikes.replace({u'Día de la semana': {0:'Lunes', 1:'Martes', 2:u'Miércoles', 3:'Jueves',
                                           4:'Viernes', 5:u'Sábado', 6:'Domingo'}}, inplace=True)

berri_bikes.boxplot(column=u'Berri 1', by=u'Día de la semana',
                    positions= [7, 4, 1, 2, 3, 6, 5])         # Las posiciones es para mantener el orden alfabético



Estas gráficas se conocen como *Box Plots* y son muy útiles para visualizar la distribución de datos. La línea roja representa la mediana (o quantil 50) la cajita es el rango en que se encuentran el 90% de los valores, y las líneas representan los valores mínimos y máximos.

### Ejercicio: Responde a la pregunta ¿Que tanto sale la gente a andar en bicicleta por Montreal por més del año en el 2012?

In [ ]:
# Agrega todo tu código aqui, no es necesario hacerlo por partes